In [1]:
import pandas as pd
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

In [2]:
#Read in CSV Files
csv_file1 = "nflstatistics/Basic_Stats.csv"
basic_statistics_df = pd.read_csv(csv_file1)
basic_statistics_df.head()

,Age,Birth Place,Birthday,College,Current Status,Current Team,Experience,Height (inches),High School,High School Location,Name,Number,Player Id,Position,Weight (lbs),Years Played
0,NaN,"Grand Rapids , MI",5/23/1921,Notre Dame,Retired,NaN,3 Seasons,71.0,NaN,NaN,"Evans, Fred",NaN,fredevans/2513736,NaN,185.0,1946 - 1948
1,NaN,"Dayton , OH",12/21/1930,Dayton,Retired,NaN,1 Season,70.0,NaN,NaN,"Raiff, Jim",NaN,jimraiff/2523700,NaN,235.0,1954 - 1954
2,56.0,"Temple , TX",9/11/1960,Louisiana Tech,Retired,NaN,1 Season,74.0,NaN,NaN,"Fowler, Bobby",NaN,bobbyfowler/2514295,NaN,230.0,1985 - 1985
3,30.0,"New Orleans , LA",9/30/1986,LSU,Retired,NaN,5 Seasons,73.0,NaN,NaN,"Johnson, Quinn",NaN,quinnjohnson/79593,NaN,255.0,2009 - 2013
4,25.0,"Detroit , MI",3/31/1992,Central Michigan,Active,Pittsburgh Steelers,3rd season,77.0,Clintondale HS,"Clinton Twp.,Macomb Co., MI","Walton, L.T.",96.0,l.t.walton/2552444,DE,305.0,NaN


In [3]:
csv_file2 = "Madden_Ultimate_Team.csv"
Madden_Ultimate_Team = pd.read_csv(csv_file2)
Madden_Ultimate_Team.head()

,Unnamed: 0,Name,Position,Team,Height,Weight,Overall,Card_set,Player_page,SPD,...,PRC,MCV,ZCV,PRS,KPW,KAC,KR,Xbox_price,Ps4_price,Quick_sell
0,0,Mean Joe Greene,DT,Pittsburgh Steelers,76,275,94,Night Guardians,/19/players/51534-mean-joe-greene,77.0,...,91.0,24.0,35.0,23.0,35.0,34.0,20.0,862000,870000,9000
1,1,Calvin Johnson,WR,Detroit Lions,77,236,94,MUT Monsters,/19/players/51501-calvin-johnson,91.0,...,46.0,42.0,65.0,38.0,20.0,20.0,47.0,905000,950000,9000
2,2,Night Train Lane,CB,Detroit Lions,73,194,94,Night Guardians,/19/players/52216-night-train-lane,89.0,...,93.0,89.0,90.0,92.0,35.0,33.0,65.0,1490000,1500000,500000
3,3,Ronnie Lott,FS,San Francisco 49ers,72,203,94,Team Diamonds,/19/players/55100-ronnie-lott,89.0,...,94.0,81.0,92.0,80.0,22.0,20.0,35.0,903000,961000,9000
4,4,Anthony Munoz,LT,Cincinnati Bengals,78,280,94,Legends,/19/players/52203-anthony-munoz,69.0,...,15.0,12.0,15.0,10.0,21.0,20.0,10.0,860000,780000,500000


In [4]:
pID = basic_statistics_df['Player Id'].str.split('/', expand=True)
pID.drop(labels=1, axis=1, inplace=True)
pID.rename(columns = {0:'ID'}, inplace=True)
player_data = pd.concat([pID,basic_statistics_df], axis=1, join='inner')
player_data.rename({'ID':'id', 'Age':'age','Name': 'name', 'Number':'number', 'Position': 'position', 'Birth Place': 'birth_place', 'College': 'college', 'Current Status': 'current_status', 'Current Team': 'current_team', 'Experience': 'experience', 'High School': 'high_school', 'Height (inches)': 'height', 'Weight (lbs)': 'weight_lbs', 'Years Played': 'years_played'},axis=1, inplace=True)
pfr_df = player_data.filter(items=['id', 'age', 'position', 'birth_place', 'college', 'current_status', 'current_team', 'experience', 'height', 'high_school', 'name', 'number', 'weight_lbs', 'years_played'])
pfr_df.head()

,id,age,position,birth_place,college,current_status,current_team,experience,height,high_school,name,number,weight_lbs,years_played
0,fredevans,NaN,NaN,"Grand Rapids , MI",Notre Dame,Retired,NaN,3 Seasons,71.0,NaN,"Evans, Fred",NaN,185.0,1946 - 1948
1,jimraiff,NaN,NaN,"Dayton , OH",Dayton,Retired,NaN,1 Season,70.0,NaN,"Raiff, Jim",NaN,235.0,1954 - 1954
2,bobbyfowler,56.0,NaN,"Temple , TX",Louisiana Tech,Retired,NaN,1 Season,74.0,NaN,"Fowler, Bobby",NaN,230.0,1985 - 1985
3,quinnjohnson,30.0,NaN,"New Orleans , LA",LSU,Retired,NaN,5 Seasons,73.0,NaN,"Johnson, Quinn",NaN,255.0,2009 - 2013
4,l.t.walton,25.0,DE,"Detroit , MI",Central Michigan,Active,Pittsburgh Steelers,3rd season,77.0,Clintondale HS,"Walton, L.T.",96.0,305.0,NaN


In [5]:
Madden_Ultimate_Team.drop(labels='Unnamed: 0', axis=1, inplace=True)
mID = Madden_Ultimate_Team['Name'].str.replace(' ','').str.lower()
Madden_Ultimate_Team.insert(0,'ID',mID)
new_madden_df = Madden_Ultimate_Team[['ID', 'Name', 'Position', 'Overall']].copy()
new_madden_df.rename({'ID':'id', 'Name':'name', 'Position':'position', 'Overall':'overall'},axis=1, inplace=True)
madden = new_madden_df.drop_duplicates(subset='id', keep='last')
madden.head()

,id,name,position,overall
0,meanjoegreene,Mean Joe Greene,DT,94
1,calvinjohnson,Calvin Johnson,WR,94
2,nighttrainlane,Night Train Lane,CB,94
3,ronnielott,Ronnie Lott,FS,94
4,anthonymunoz,Anthony Munoz,LT,94


In [6]:
rds_connection_string = "postgres:Jack4@localhost:5432/Madden"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
madden.to_sql(name='player_info', con=engine, if_exists='append', index=False)

In [ ]:
pfr_df.to_sql(name='pro_football_reference', con=engine, if_exists='append', index=False)